<a href="https://colab.research.google.com/github/FateenAnam/Big-Data-Final-Project/blob/main/0_mrjob_basics_emr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install mrjob
!pip install mrjob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 KB 7.3 MB/s eta 0:00:00


In [2]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 77.0 MB/s eta 0:00:00


### Configure mrjob

You need to polulate a config file to allow mrjob to access or create an EMR cluster under your AWS account. You did something similar with the pandas assignment. The information can be obtained from 

An example mrjob.conf file is like:
```yaml
runners: 
    emr: 
        region: xxx
        aws_access_key_id: xxx
        aws_secret_access_key: xxx
        aws_session_token: xxx
```

An example mrjob.conf file can be found in your repository. Please replace "xxx" with your own credential information, which can be found on the AWS academy lab main page under cloud access. See the readme of the assignment.


# Upload your mrjob.conf from your machine to the colab location

In [3]:
from google.colab import files

# upload mrjob.conf under current directory
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving mrjob.conf to mrjob.conf
Saving emr.pem to emr.pem
User uploaded file "mrjob.conf" with length 647 bytes
User uploaded file "emr.pem" with length 1674 bytes


In [4]:
!chmod 400 emr.pem

# Total Word count example


In [5]:
%%file mr_word_count.py
from mrjob.job import MRJob
class MRWordFrequencyCount(MRJob):
  def mapper(self, _, line):
    yield "chars", len(line)
    yield "words", len(line.split())
    yield "lines", 1
  
  def combiner(self, key, values):
    yield key, sum(values)
    
  def reducer(self, key, values):
    yield key, sum(values)


if __name__ == '__main__': 
  MRWordFrequencyCount.run() 

Writing mr_word_count.py


## Step 1. Run MR program locally(Colab)

In [6]:
# Run locally to check that the output looks okay and your core algorithm is good.
# You should always run locally. It can be done with just a sample of the main data.
# However, in this case because the input file is small it does not matter and we run with full file.
!python mr_word_count.py -r local sample_data/README.md >output.local

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/mr_word_count.root.20230331.204646.933326
Running step 1 of 1...
job output is in /tmp/mr_word_count.root.20230331.204646.933326/output
Streaming final output from /tmp/mr_word_count.root.20230331.204646.933326/output...
Removing temp directory /tmp/mr_word_count.root.20230331.204646.933326...


In [7]:
!cat output.local

"words"	80
"lines"	19
"chars"	911


# Step 2. Run MR program on AWS EMR. 

The code below assumes that you have a S3 bucket called vandy-bigdata where a file readme.md has been saved. It also assumes there is a folder called tmp in your bucket. It also requires that you have an EMR cluster that has been created. You will have to put the correct id of the EMR Cluster in the code below.

Follow the instructions below to create an EMR Cluster for yourself. it can take up to 20 minutes to provision an EMR Cluster.  Note there are subtelties about pricing. See https://aws.amazon.com/ec2/spot/use-case/emr/ .

Further ensure that the security group for the master node has been configured to allow ssh connections.

![images/emr-security.png](https://github.com/vu-topics-in-big-data-2023/map-reduce-programming-assignment-MJRAJ01/blob/main/images/emr-security.png?raw=1)

Feel free to discuss this in the class.

In [10]:
!python mr_word_count.py -r emr s3://vandy-bigdata-1/README.md --cloud-tmp-dir=s3://vandy-bigdata-1/ --cluster-id=j-1ROJ0IHL57KD0 --conf-path mrjob.conf >output.emr

Creating temp directory /tmp/mr_word_count.root.20230331.210538.857625
uploading working dir files to s3://vandy-bigdata-1/mr_word_count.root.20230331.210538.857625/files/wd...
Copying other local files to s3://vandy-bigdata-1/mr_word_count.root.20230331.210538.857625/files/
Adding our job to existing cluster j-1ROJ0IHL57KD0
  master node is ec2-35-173-203-184.compute-1.amazonaws.com
Waiting for Step 1 of 1 (s-1QCMJ5MUEU49T) to complete...
  PENDING (cluster is RUNNING: Running step)
  RUNNING for 0:00:30
    16.3% complete
  COMPLETED
Attempting to fetch counters from logs...
Looking for step log in /mnt/var/log/hadoop/steps/s-1QCMJ5MUEU49T on ec2-35-173-203-184.compute-1.amazonaws.com...
  Parsing step log: ssh://ec2-35-173-203-184.compute-1.amazonaws.com/mnt/var/log/hadoop/steps/s-1QCMJ5MUEU49T/syslog
Counters: 60
	File Input Format Counters 
		Bytes Read=57791
	File Output Format Counters 
		Bytes Written=39
	File System Counters
		FILE: Number of bytes read=230
		FILE: Number of b

**Remember to Terminate your EMR Cluster when you are done and delete the S3 buckets that you dont need once you are done**

In [11]:
!cat output.emr

"lines"	198
"chars"	13179
"words"	1694


## What is happening

Note that we are writing a python programs using %%file mymrjob.py -- this will copy the content of the cell into a file with the same name. We then run this using the local python command and provide the command line input.


In each mrjob file a job is defined by a class that inherits from MRJob. This class contains methods that define the steps of your job. A “step” consists of a mapper, a combiner, and a reducer. All of those are optional, though you must have at least one. So you could have a step that’s just a mapper, or just a combiner and a reducer.

When you only have one step, all you have to do is write methods called mapper(), combiner(), and reducer().


The mapper() method takes a key and a value as args (in this case, the key is ignored and a single line of text input is the value) and yields as many key-value pairs as it likes. The reduce() method takes a key and an iterator of values and also yields as many key-value pairs as it likes. (In this case, it sums the values for each key, which represent the numbers of characters, words, and lines in the input.)


The final required component of a job file is these two lines at the end of the file, every time:


```
if __name__ == '__main__':
  MRWordCounter.run() # where MRWordCounter is your job class
```

These lines pass control over the command line arguments and execution to mrjob.

You invoke it using the python command line



```
!python mymrjob.py -r emr s3://<your data file> --cloud-tmp-dir=s3://<directory for storing runtime tmp files> --cluster-id=<emr cluster ID> --conf-path <path of mrjob.conf> > output
```

read the https://buildmedia.readthedocs.org/media/pdf/mrjob/latest/mrjob.pdf for more details. Specifically read section 1.4.2 in the manual about the protocols.
